In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchsummary
from IPython.display import clear_output
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import os


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Model

In [ ]:
class VisualEncoderModel(nn.Module):
    def __init__(self):
        super(VisualEncoderModel, self).__init__()
        self.block1 = nn.Conv2d(in_channels=channels, 
                              out_channels=channels,
                              kernel_size=kernel_size,
                              padding=kernel_size//2)
        
        self.block2 = TinyImageNetResidualBlock(channels=64, kernel_size=5)
#         self.block2 = TinyImageNetResidualSqueezeBlock(channels=64, 
#                                                        kernel_size=5,
#                                                        r=16)
    
        self.sq1 = TinyImageNetResidualBlock(channels=64, kernel_size=3)
#         self.sq1 = TinyImageNetResidualSqueezeBlock(channels=64, 
#                                                     kernel_size=3, 
#                                                     r=16)
        
        self.block3 = TinyImageNetBlockWithPool(in_channels=64,
                                                out_channels=128,
                                                kernel_size=3,
                                                pool_size=2)
        
        self.block4 = TinyImageNetResidualBlock(channels=128, kernel_size=3)
#         self.block4 = TinyImageNetResidualSqueezeBlock(channels=128, 
#                                                        kernel_size=3,
#                                                        r=16)
        
        self.sq2 = TinyImageNetResidualBlock(channels=128, kernel_size=3)
#         self.sq2 = TinyImageNetResidualSqueezeBlock(channels=128,
#                                                     kernel_size=3,
#                                                     r=16)
        
        self.block5 = TinyImageNetBlockWithPool(in_channels=128, 
                                                out_channels=256, 
                                                kernel_size=3, 
                                                pool_size=2)
        
        self.block6 = TinyImageNetResidualBlock(channels=256, kernel_size=3)
#         self.block6 = TinyImageNetResidualSqueezeBlock(channels=256, 
#                                                        kernel_size=3, 
#                                                        r=16)
        
        self.block7 = TinyImageNetBlockWithPool(in_channels=256,
                                                out_channels=512, 
                                                kernel_size=3,
                                                pool_size=1)
        
        self.block8 = TinyImageNetBlockWithPool(in_channels=512,
                                                out_channels=1024,
                                                kernel_size=1,
                                                pool_size=1)
        
        self.average_pool = nn.AdaptiveAvgPool2d(1)
        self.dense_logits = nn.Linear(1024, 50)
        
    def forward(self, input):
        h = self.block5(self.sq2(self.block4(self.block3(self.sq1(self.block2(self.block1(input)))))))
        h = self.average_pool(self.block8(self.block7(self.block6(h)))).view(input.shape[0], -1)
        return self.dense_logits(h)
